In [1]:
### This notebook implement DEseq from count data and draw candidates from cutoffs

In [1]:
import pandas
from os.path import join
from os import listdir
import pandas as pd
# from tqdm import tqdm
from collections import Counter
import re
import math

from Bio import SeqIO
from Bio.Seq import MutableSeq,Seq

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# count_path = '../data/output/library_alignment'
count_path = '../data/sequencing/Library/read_count'
metadata = pd.read_csv(join(count_path, 'metadata.csv'), index_col = 0)

In [3]:
def calc_L2FC(df,lib1,lib2):
    l2fc1 = np.log2((df[lib1]/df[lib1].sum()*1000000)+1)
    l2fc2 = np.log2((df[lib2]/df[lib2].sum()*1000000)+1)
    l2fc = l2fc2-l2fc1
    return(l2fc)

def calc_L2RD(df,lib1):
    l2fc1 = np.log2((df[lib1]/df[lib1].sum()*1000000)+1)
    return(l2fc1)

def calc_threshold(df_column):
    m_3std = (df_column.mean()-3*df_column.std(),df_column.mean()+3*df_column.std())
    return(m_3std)

def filter_sg(df, lib):
    m_3std = calc_threshold(df[lib])
    filter_idx = df[(df[lib] > m_3std[0]) & (df[lib] < m_3std[1])].index
    return(filter_idx)   

In [4]:
cell_line = 'K562'; be ='ABE'; baseline = 'DN1'; endpoint = 'D0'

baseline_df = pd.read_csv(join(count_path, f'{cell_line}_{baseline}_{be}_df.csv'), index_col = 0)
endpoint_df = pd.read_csv(join(count_path, f'{cell_line}_{endpoint}_{be}_df.csv'), index_col = 0)

test_df = pd.concat([baseline_df, endpoint_df], axis =1)

In [5]:
filter_libs = [i for i in test_df.columns if 'DN1' in i]

In [6]:
df = test_df.copy()
lib1 = filter_libs[0]
lib2 = filter_libs[1]
df_rpm = pd.DataFrame([calc_L2RD(df, lib1), \
                       calc_L2RD(df, lib2)]).T
tsd1 = filter_sg(df_rpm,lib1)
tsd2 = filter_sg(df_rpm,lib2)
index = tsd1.intersection(tsd2)
test_df = test_df.loc[index,:]

In [7]:
test_df.to_csv(join('../data/sequencing/Library/filtered_read_count', f'{cell_line}_{be}_{baseline}_{endpoint}.csv'))

In [8]:
test_df['index'] = test_df.index
test_df['categ'] = test_df['index'].apply(lambda x: x.split('_')[0])

In [9]:
cols = [i for i in test_df.columns if i.startswith('count')]
clinical_df = pd.DataFrame(cols, columns = ['Day'])

In [10]:
clinical_df

,Day
0,count_KC1_DN1
1,count_KC2_DN1
2,count_KC1_D0_CBE
3,count_KC1_D0_nSpG
4,count_KC2_D0_CBE
5,count_KC2_D0_nSpG


In [11]:
cols = [i for i in test_df.columns if i.startswith('count')]
clinical_df = pd.DataFrame(index = cols, columns = ['Day'])
clinical_df['Day'] = [i.split('_')[2] for i in cols]

In [12]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [13]:
clinical_df

,Day
count_KC1_DN1,DN1
count_KC2_DN1,DN1
count_KC1_D0_CBE,D0
count_KC1_D0_nSpG,D0
count_KC2_D0_CBE,D0
count_KC2_D0_nSpG,D0


In [14]:
library = 'CBE'; 

baseline_index = [i for i in clinical_df.index if i.endswith(baseline)]
library_index = [i for i in clinical_df.index if i.endswith(library)]
sel_index = baseline_index+library_index

In [15]:
sel_index

['count_KC1_DN1', 'count_KC2_DN1', 'count_KC1_D0_CBE', 'count_KC2_D0_CBE']

In [37]:
# Testing the ABE or the control nSpG library
anlyzeP = '../data/output/DESeq'
dds = DeseqDataSet(counts=test_df.T.loc[sel_index,:],metadata = clinical_df.loc[sel_index,:],
                       design_factors='Day',
                       refit_cooks=True)
dds.deseq2() #Fit dispersion and normlization
stat_res = DeseqStats(dds, contrast=('Day', endpoint, baseline))
stat_res.summary() 
stat_res.results_df.to_csv(join(anlyzeP,f'DESeq2_{cell_line}_{be}_{library}_{endpoint}_{baseline}.csv'))

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 1.30 seconds.

Fitting dispersion trend curve...
... done in 0.25 seconds.

/storage/group/epo2/default/yur97/conda_envs/deseq2/lib/python3.8/site-packages/pydeseq2/dds.py:425: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
Fitting MAP dispersions...
... done in 1.44 seconds.

Fitting LFCs...
... done in 0.78 seconds.

Refitting 0 outliers.

Running Wald tests...


Log2 fold change & Wald test p-value: Day D14 vs DN1
                    baseMean  log2FoldChange     lfcSE      stat    pvalue  \
SYN_CLP1_0       1945.331373       -0.013813  0.373973 -0.036936  0.970536   
SYN_CLP1_1       3928.610715        0.405055  0.251582  1.610029  0.107392   
SYN_CLP1_2       1188.032708       -0.404028  0.432284 -0.934634  0.349977   
SYN_CLP1_3        761.516829       -0.072677  0.500663 -0.145162  0.884583   
SYN_CLP1_4       1686.519526       -0.132085  0.353544 -0.373603  0.708699   
...                      ...             ...       ...       ...       ...   
Control_NT_1792  2893.796384        0.434050  0.289214  1.500789  0.133410   
Control_NT_1793  1184.149781        0.026528  0.402771  0.065865  0.947486   
Control_NT_1794  1093.183194       -0.456190  0.447151 -1.020214  0.307627   
Control_NT_1795  1468.753588        0.099222  0.360585  0.275168  0.783187   
Control_NT_1796  1524.625376        0.173527  0.372081  0.466369  0.640951   

          

... done in 0.56 seconds.

